In [2]:
import pynmea2
import numpy as np
import time
import pandas as pd
import datetime
import os
import chardet

# Dash
import dash
from dash import dcc, html
import folium
from dash.dependencies import Input, Output, State


def detect_encoding(file_path):
    with open(file_path, 'rb') as file:
        raw_data = file.read()
        result = chardet.detect(raw_data)
        encoding = result['encoding']
        if encoding == 'utf-8':
            replacer = 'QQ5±'
        else:
            replacer = 'QQ5Â±'
        return encoding, replacer
    
def gps_data_to_df(gps_data_path):
    
    if type(gps_data_path) is not list:
        gps_data_path = [gps_data_path]
    
    for gps_dir in gps_data_path:
        files = os.listdir(gps_dir)
        df_gpgga = pd.DataFrame(columns=["timestamp", "message_id", "utc", "lat", "lon", "position_accuracy", "altitude_above_sea"])
        df_gphdt = pd.DataFrame(columns=["timestamp", "message_id", "heading_degrees"])
        for gps_file in files:
            gps_file = os.path.join(gps_dir, gps_file)
            encoding, _ = detect_encoding(gps_file)
            with open(gps_file, "rt", encoding=encoding) as gps_serial_data:
                for line in gps_serial_data:
                    try:
                        if 'GPGGA' in line.rstrip():
                            gpgga = pynmea2.parse(line.split('$')[1])

                            # Extract the desired information
                            timestamp = line.split('$')[0]
                            timestamp = timestamp.strip("[] ")
                            timestamp = datetime.datetime.strptime(timestamp, "%Y-%m-%d %H:%M:%S.%f")
                            timestamp = timestamp.timestamp()
                            message_id = gpgga.sentence_type
                            utc = gpgga.timestamp.replace(tzinfo=None)
                            lat = gpgga.latitude
                            lon = gpgga.longitude
                            position_accuracy = gpgga.gps_qual
                            altitude_above_sea = gpgga.altitude

                            df_gpgga = pd.concat([df_gpgga, pd.Series({"timestamp": timestamp,
                                                                       "message_id": message_id,
                                                                       "utc": utc,
                                                                       "lat": lat,
                                                                       "lon": lon,
                                                                       "position_accuracy": position_accuracy, 
                                                                       "altitude_above_sea": altitude_above_sea}).to_frame().T], ignore_index=True)

                        if 'GPHDT' in line.rstrip():
                            gphdt = pynmea2.parse(line.split('$')[1])

                            # Extract the desired information
                            timestamp = line.split('$')[0]
                            timestamp = timestamp.strip("[] ")
                            timestamp = datetime.datetime.strptime(timestamp, "%Y-%m-%d %H:%M:%S.%f")
                            timestamp = timestamp.timestamp()
                            message_id = gphdt.sentence_type
                            heading_degrees = float(gphdt.heading) * np.pi /180

                            df_gphdt = pd.concat([df_gphdt, pd.Series({"timestamp": timestamp,
                                                                       "message_id": message_id,
                                                                       "heading_degrees": heading_degrees}).to_frame().T], ignore_index=True)

                    except Exception as e:
                        # Handle parse errors, if any
                        print(e)
                        pass
    
    return df_gpgga, df_gphdt

def radar_data_to_df(radar_data_path):
    
    if type(radar_data_path) is not list:
        radar_data_path = [radar_data_path]
        
    df = pd.DataFrame(columns=["timestamp", "message_id", 'target_number', 'distance', 'bearing', 'brg_ref', 'speed', 'cog', 'cog_unit', 'dist_cpa', 'time_cpa', 'dist_unit', 'name', 'status', 'reference', 'utc', 'acquisition'])
    for radar_data in radar_data_path:
        encoding, replacement_string = detect_encoding(radar_data)
        with open(radar_data, "rt", encoding=encoding) as radar_serial_data:
            for line in radar_serial_data:
                try:

                    line = line.replace(replacement_string, '$RATTM,')
                    if 'RATTM' in line.rstrip():
                        rattm = pynmea2.parse(line.split('$')[1])

                        # Extract the desired information
                        timestamp = line.split('$')[0]
                        timestamp = timestamp.strip("[] ")
                        timestamp = datetime.datetime.strptime(timestamp, "%Y-%m-%d %H:%M:%S.%f")
                        timestamp = timestamp.timestamp()
                        message_id = rattm.sentence_type
                        utc = rattm.timestamp.replace(tzinfo=None)
                        target_number = rattm.target_number
                        distance = rattm.distance
                        bearing = rattm.bearing
                        brg_ref = rattm.brg_ref
                        speed = rattm.speed
                        cog = rattm.cog
                        cog_unit = rattm.cog_unit
                        dist_cpa = rattm.dist_cpa
                        time_cpa = rattm.time_cpa
                        dist_unit = rattm.dist_unit
                        name = rattm.name
                        status = rattm.status
                        reference = rattm.reference
                        utc = rattm.timestamp
                        acquisition = rattm.acquisition

                        df = pd.concat([df, pd.Series({"timestamp": timestamp,
                                                       "message_id": message_id,
                                                       'target_number': target_number,
                                                       'distance': distance,
                                                       'bearing': bearing, 
                                                       'brg_ref' : brg_ref, 
                                                       'speed' : speed, 
                                                       'cog' : cog, 
                                                       'cog_unit' : cog_unit, 
                                                       'dist_cpa' : dist_cpa, 
                                                       'time_cpa' : time_cpa, 
                                                       'dist_unit' : dist_unit, 
                                                       'name' : name, 
                                                       'status' : status, 
                                                       'reference' : reference, 
                                                       'utc' : utc, 
                                                       'acquisition' : acquisition}).to_frame().T], ignore_index=True)

                except Exception as e:
                    # Handle parse errors, if any
                    print(e)
                    pass
    return df

def create_combined_dataframe(df_rattm, df_gpgga, df_gphdt):

    closest_values = []

    timestamps_gpgga = pd.to_datetime(df_gpgga['timestamp'], unit='s')
    timestamps_gphdt = pd.to_datetime(df_gphdt['timestamp'], unit='s')


    # Iterate over each timestamp in DataFrame B
    for timestamp_b in df_rattm['timestamp']:
        timestamp_b = pd.to_datetime(timestamp_b, unit='s')

        # Calculate the time difference between each timestamp in B and all timestamps in A
        time_diff_gpgga = np.abs(timestamps_gpgga - timestamp_b)
        time_diff_gphdt = np.abs(timestamps_gphdt - timestamp_b)

        # Find the index of the timestamp in A with the minimum time difference
        closest_index_gpgga = time_diff_gpgga.idxmin()
        closest_index_gphdt = time_diff_gphdt.idxmin()

        # Get the corresponding value from A using the closest index
        closest_lat = df_gpgga.loc[closest_index_gpgga, 'lat']
        closest_long = df_gpgga.loc[closest_index_gpgga, 'lon']
        closest_heading = float(df_gphdt.loc[closest_index_gphdt, 'heading_degrees'])

        # Add the closest value to the list
        closest_values.append([closest_lat, closest_long, closest_heading])

    gps_df = pd.DataFrame(np.array(closest_values), columns=['lat_ref', 'long_ref', 'bearing_ref'])
    df_comb = pd.concat([df_rattm, gps_df], axis=1)
    return df_comb

def calculate_lat_and_long(row, nautical_miles_per_kilometer, earth_radius):
    
    if row.status == 'T':
        
        # Finding range for target
        r = float(row.distance) * nautical_miles_per_kilometer # km

        bearing_rad = np.radians(float(row.bearing)) # rad

        # Coordinate calculation
        d = r / nautical_miles_per_kilometer  # Distance to object km

        Ad = d / earth_radius  # Angular distance i.e d/R (nautical miles) # (rad)

        lat1 = np.radians(row.lat_ref) #rad
        long1 = np.radians(row.long_ref) #rad

        #            #deg       rad          (rad)         (rad)          (rad)         #rad          #rad
        tmp_la = np.degrees(np.arcsin(np.sin(lat1) * np.cos(Ad) + np.cos(lat1) * np.sin(Ad) * np.cos(bearing_rad)))
                    #deg     (rad)     (rad)             (rad)              (rad)         (rad)
        tmp_lo = np.degrees(long1 + np.arctan2((np.sin(bearing_rad) * np.sin(Ad) * np.cos(lat1)),
                               (np.cos(Ad) - np.sin(lat1) * np.sin(np.radians(tmp_la)))))
                                    # (rad)        (rad)           (deg!)--> (rad)
    else:
        tmp_lo = None
        tmp_la = None
    
    return pd.Series([tmp_la, tmp_lo], index=['calculated_lat', 'calculated_long'])

In [18]:
gps_data_path = [r"C:\Projects\sigray\logs\logs_20230512\log_path_demo\2023_05_16_13_30\serial1", r"C:\Projects\sigray\logs\logs_20230512\log_path_demo\2023_05_16_11_30\serial1"]
radar_data_path = [r"C:\Projects\sigray\logs\logs_20230512\log_path_demo\2023_05_16_13_30\serial0\complete_log.log", r"C:\Projects\sigray\logs\logs_20230512\log_path_demo\2023_05_16_11_30\serial0\complete_log.log"]
df_gpgga, df_gphdt = gps_data_to_df(gps_data_path)
df_rattm = radar_data_to_df(radar_data_path)
df_comb = create_combined_dataframe(df_rattm, df_gpgga, df_gphdt)
nautical_miles_per_kilometer = 1852 / 1000
earth_radius = 6371  # Radius of Earth
df_comb[['calculated_lat', 'calculated_long']] = df_comb.apply(calculate_lat_and_long, args=(nautical_miles_per_kilometer, earth_radius,), axis=1)
df_comb.head()

('could not parse data', 'GPHDT')
unconverted data remains: ] THø
time data 'á' does not match format '%Y-%m-%d %H:%M:%S.%f'
('could not parse data', 'GPHDT')
Geographic coordinate value '5' is not valid DDDMM.MMM
unconverted data remains: ] 0,UHø
time data 'ø' does not match format '%Y-%m-%d %H:%M:%S.%f'
Geographic coordinate value '01126.' is not valid DDDMM.MMM
unconverted data remains: ] ,UHø
Geographic coordinate value '011' is not valid DDDMM.MMM
Geographic coordinate value '01' is not valid DDDMM.MMM
unconverted data remains: ] 3KUHø
float() argument must be a string or a real number, not 'NoneType'
unconverted data remains: ] ,UHø
Geographic coordinate value '01126.' is not valid DDDMM.MMM
Geographic coordinate value '5814' is not valid DDDMM.MMM
float() argument must be a string or a real number, not 'NoneType'
str.replace() takes no keyword arguments
Geographic coordinate value '581' is not valid DDDMM.MMM
('could not parse data', 'GPHDT')
float() argument must be a string or

,timestamp,message_id,target_number,distance,bearing,brg_ref,speed,cog,cog_unit,dist_cpa,...,name,status,reference,utc,acquisition,lat_ref,long_ref,bearing_ref,calculated_lat,calculated_long
0,1684235602.800136,TTM,0,0.097,52.03,T,None,None,T,None,...,,Q,R,11:13:22+00:00,M,58.24995,11.445383,0.558505,NaN,NaN
1,1684235605.150184,TTM,0,0.097,52.03,T,None,None,T,None,...,,Q,R,11:13:24+00:00,M,58.24995,11.445383,0.558505,NaN,NaN
2,1684235606.86019,TTM,0,0.097,51.75,T,None,None,T,None,...,,Q,R,11:13:26+00:00,M,58.24995,11.445383,0.558505,NaN,NaN
3,1684235608.800203,TTM,0,0.097,51.67,T,None,None,T,None,...,,Q,R,11:13:28+00:00,M,58.24995,11.445383,0.558505,NaN,NaN
4,1684235612.770118,TTM,0,0.097,51.51,T,None,None,T,None,...,,Q,R,11:13:32+00:00,M,58.24995,11.445383,0.558505,NaN,NaN


In [61]:
def create_map_object(init_zoom, radar_init_lat, radar_init_long):
    map_object = folium.Map([radar_init_lat, radar_init_long], zoom_start=init_zoom, tiles="cartodbpositron")
    folium.Marker(
        location=[radar_init_lat, radar_init_long],
        popup="Boat radar",
    ).add_to(map_object)

    html_string = '''
    <!doctype html>
    <html>
    <head>
        <meta charset="utf-8">
        <title>Map</title>
        <meta name="viewport" content="width=device-width, initial-scale=1">
        <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.7/css/bootstrap.min.css">
    </head>
    <body>
        <div class="container-fluid">
            <div class="row-fluid">
                <div class="span12">
                    <div id="map"></div>
                </div>
            </div>
        </div>
        <script>
            {{ script }}
        </script>
    </body>
    </html>
    '''

    folium_map = folium.Map(location=[radar_init_lat, radar_init_long], zoom_start=init_zoom)
    folium.Element(html_string).add_to(folium_map)

    # Display initial map
    map_name = 'map.html'
    folium_map.save(map_name)
    return map_object


base_lat, base_long = df_comb.iloc[0][['lat_ref', 'long_ref']]
init_zoom = 14
interval_time = 2 * 1000  # milliseconds
simulation_speed = 500

nautical_miles_per_kilometer = 1852 / 1000
earth_radius = 6371  # Radius of Earth

m = create_map_object(init_zoom, base_lat, base_long)
df = df_comb.copy()

# Create a Dash app
app = dash.Dash(__name__)
# Run the app
app.layout = html.Div([
    html.Iframe(id='map', srcDoc=m._repr_html_(), width='100%', height='1000'),
    dcc.Store(id='stored_targets_prev', data=[]),
    dcc.Store(id='stored_targets_hist', data=[]),
    dcc.Interval(
        id="interval",
        interval=simulation_speed,
        n_intervals=0
    )
])

# Define a callback function to update the map
@app.callback(Output("map", "srcDoc"),
              Output('stored_targets_prev', 'data'),
              Output('stored_targets_hist', 'data'),
              Input("interval", "n_intervals"),
              State('stored_targets_prev', 'data'),
              State('stored_targets_hist', 'data'))
def update_map(n, old_targets, hist_targets):
    
    # Current interval
    interval_seconds = n * interval_time / 1000
    
    # Calculate the start and end timestamps based on the interval
    start_time = df['timestamp'].iloc[0] + (interval_seconds - interval_time / 1000)
    end_time = df['timestamp'].iloc[0] + interval_seconds
    
    # Add new markers to the Folium map object
    new_m = create_map_object(init_zoom, base_lat, base_long)
    
    filtered_df = df[(df['timestamp'] >= start_time) & (df['timestamp'] <= end_time)]
    filtered_df = filtered_df.dropna()
    targets = []

    # New targets
    for target_nbr, lat, long in zip(filtered_df.target_number, filtered_df.calculated_lat, filtered_df.calculated_long):
        color_map = {0: "red", 1: "blue", 2: "green", 3: "purple", 4: 'orange', 5: 'darkred', 6: 'lightred', 7: 'beige',
                     8: 'darkblue', 9: 'darkgreen', 10: 'pink'}
        color = color_map[target_nbr]
        folium.CircleMarker(location=[lat, long],
                            radius=3,
                            color="black",
                            opacity=0.8,
                            weight=1,
                            fill=True,
                            fill_color=color,
                            fill_opacity=0.8,
                            ).add_to(new_m)
        targets.append((target_nbr, lat, long))

    # Previous targets
    for old_target in old_targets:
        target_nbr, lat, long = old_target
        color_map = {0: "red", 1: "blue", 2: "green", 3: "purple", 4: 'orange', 5: 'darkred', 6: 'lightred', 7: 'beige',
                     8: 'darkblue', 9: 'darkgreen', 10: 'pink'}
        color = color_map[target_nbr]
        folium.CircleMarker(location=[lat, long],
                            radius=3,
                            color="black",
                            opacity=0.5,
                            weight=1,
                            fill=True,
                            fill_color=color,
                            fill_opacity=0.5,
                            ).add_to(new_m)

    # Previous targets
    for hist_target in hist_targets:
        target_nbr, lat, long = hist_target
        color_map = {0: "red", 1: "blue", 2: "green", 3: "purple", 4: 'orange', 5: 'darkred', 6: 'lightred', 7: 'beige',
                     8: 'darkblue', 9: 'darkgreen', 10: 'pink'}
        color = color_map[target_nbr]
        folium.CircleMarker(location=[lat, long],
                            radius=3,
                            color="black",
                            opacity=0.2,
                            weight=1,
                            fill=True,
                            fill_color=color,
                            fill_opacity=0.2,
                            ).add_to(new_m)

    # Convert the Folium map object to HTML
    html_map = new_m._repr_html_()

    # Return the HTML as a child of the Dash Map component
    return html_map, targets, old_targets


# Run the app
if __name__ == '__main__':

    app.run_server()


Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8050
Press CTRL+C to quit
127.0.0.1 - - [26/May/2023 15:05:49] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2023 15:05:50] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2023 15:05:50] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2023 15:05:50] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2023 15:05:50] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2023 15:05:51] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2023 15:05:51] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2023 15:05:51] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2023 15:05:52] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2023 15:05:52] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2023 15:05:53] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2023 15:05:53] "POST /_dash-update-component HT